# Logistic Regression Demonstration.

This code taken from https://machinelearningmastery.com/building-a-logistic-regression-classifier-in-pytorch/ by Jason Brownlee

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# loading training data
train_dataset = datasets.MNIST(root='./data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)
#loading test data
test_dataset = datasets.MNIST(root='./data', 
                              train=False, 
                              transform=transforms.ToTensor())



print("number of training samples: " + str(len(train_dataset)) + "\n" +
      "number of testing samples: " + str(len(test_dataset)))


print("number of training samples: " + str(len(train_dataset)) + "\n" +
      "number of testing samples: " + str(len(test_dataset)))

print("label of the first taining sample: ", train_dataset[0][1])
print("label of the second taining sample: ", train_dataset[1][1])

In [ ]:
img_5 = train_dataset[0][0].numpy().reshape(28, 28)

img_0 = train_dataset[1][0].numpy().reshape(28, 28)


plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img_5, cmap='gray')

plt.subplot(1,2,2)
plt.imshow(img_0, cmap='gray')
plt.show()

In [ ]:
from torch.utils.data import DataLoader
 
# load train and test data samples into dataloader
batach_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batach_size, shuffle=True) 
test_loader = DataLoader(dataset=test_dataset, batch_size=batach_size, shuffle=False)

# Define the logistic regression model

This is using a different method for specifying neural network models.  Here we explicitly construct a Module object and specify the forward pass by applying each of the network layers in turn. 

Also, notice that this network is using a sigmoid output instead of a softmax output.  This is because the cross-entropy loss function already computes a softmax when evaluating the loss function.  See more [here](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

In [ ]:
class LogisticRegression(torch.nn.Module):    
    # build the constructor
    def __init__(self, n_inputs, n_outputs):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(n_inputs, n_outputs)
    # make predictions
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

# Define the training function.

In [ ]:
def train_network(model, train_loader, test_loader, lr=0.001, epochs=50):

    # defining the optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    # defining Cross-Entropy loss
    criterion = torch.nn.CrossEntropyLoss()
 
    Loss = []
    acc = []
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad() # <--- here we are zeroing the grad before we start computing the 
            outputs = model(images.view(-1, 28*28)) # <-- Note! Magic numbers, we are flattening our image
            loss = criterion(outputs, labels)
            # Loss.append(loss.item())
            loss.backward()
            optimizer.step()
        Loss.append(loss.item())
        correct = 0
        for images, labels in test_loader:
            outputs = model(images.view(-1, 28*28))
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum()
        accuracy = 100 * (correct.item()) / len(test_dataset)
        acc.append(accuracy)
        print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))
    
    return Loss, acc

In [ ]:
# instantiate the model
n_inputs = 28*28 # makes a 1D vector of 784
n_outputs = 10

log_regr = LogisticRegression(n_inputs, n_outputs)
log_regr_loss, log_regr_acc = train_network(log_regr, train_loader, test_loader)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.plot(log_regr_loss)
plt.xlabel("no. of epochs")
plt.ylabel("total loss")
plt.title("Loss")

plt.subplot(1,2,2)
plt.plot(log_regr_acc)
plt.xlabel("no. of epochs")
plt.ylabel("total accuracy")
plt.title("Accuracy")
plt.show()

# Building a sequential model

This technique uses the Sequential object we saw in class.  Because ```CrossEntropyLoss``` contains a softmax, we could remove the sigmoid output from this network, but then when we wanted to use the network, we would need to apply the softmax ourself.

In [ ]:
seq_log_regr = nn.Sequential(
    nn.Linear(n_inputs, n_outputs),
    nn.Sigmoid(),
)

seq_loss, seq_acc = train_network(seq_log_regr, train_loader, test_loader)

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.plot(seq_loss)
plt.xlabel("no. of epochs")
plt.ylabel("total loss")
plt.title("Loss")

plt.subplot(1,2,2)
plt.plot(seq_acc)
plt.xlabel("no. of epochs")
plt.ylabel("total accuracy")
plt.title("Accuracy")
plt.show()